In [1]:
### This script will automatically detect ZDR arcs (and KDP feet) in WSR-88D radar data
import matplotlib.pyplot as plt
import pyart
import numpy as np
import numpy.ma as ma
from metpy.units import atleast_1d, check_units, concatenate, units
from matplotlib.patches import PathPatch
from matplotlib.path import Path
from siphon.radarserver import RadarServer
#rs = RadarServer('http://thredds-aws.unidata.ucar.edu/thredds/radarServer/nexrad/level2/S3/')
#rs = RadarServer('http://thredds.ucar.edu/thredds/radarServer/nexrad/level2/IDD/')
from datetime import datetime, timedelta
from siphon.cdmr import Dataset
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.io.shapereader import Reader
from cartopy.feature import ShapelyFeature
from metpy.units import atleast_1d, check_units, concatenate, units
from shapely.geometry import polygon as sp
import pyproj 
import shapely.ops as ops
from shapely.ops import transform
from shapely.geometry.polygon import Polygon
from functools import partial
from shapely import geometry
import netCDF4
from scipy import ndimage as ndi
#from skimage.feature import peak_local_max
#from skimage import data, img_as_float
from pyproj import Geod
#from metpy.calc import get_wind_dir, get_wind_speed, get_wind_components
import matplotlib.lines as mlines
import pandas as pd
import scipy.stats as stats
import csv
import pickle
#from sklearn.ensemble import RandomForestClassifier
import nexradaws
import os
import shutil


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



C:\Users\matts\AppData\Local\Continuum\anaconda3\envs\spork2020\lib\site-packages\siphon\cdmr\ncStream_pb2.py:19: DeprecationWarning: Call to deprecated create function FileDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  DESCRIPTOR = _descriptor.FileDescriptor(
C:\Users\matts\AppData\Local\Continuum\anaconda3\envs\spork2020\lib\site-packages\siphon\cdmr\ncStream_pb2.py:33: DeprecationWarning: Call to deprecated create function EnumValueDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _descriptor.EnumValueDescriptor(
C:\Users\matts\AppData\Local\Continuum\anaconda3\envs\spork2020\lib\site-packages\siphon\cdmr\ncStream_pb2.py:37: DeprecationWarning: Call to deprecated create function EnumValueDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find de

In [ ]:
#Year of start time
yearstm = np.asarray([2013, 2013, 2012, 2012, 2013, 2012])
#Month of case start time
monthstm = np.asarray([5, 4, 2, 3, 1, 5])
#Day of case start time
daystm = np.asarray([20, 18, 29, 2, 29, 30])
#Hour of case start time (in UTC)
hourstm = np.asarray([20, 0, 0, 23, 21, 23])
#Minute of case start time
start_minstm = np.asarray([0, 0, 30, 0, 0, 0])
#Duration of analysis period, in hours
durationstm = np.asarray([0.1, 0.1, 0.1, 0.1, 0.1, 0.1])
#WSR-88D radar site
stationstm = ['KTLX', 'KFDR', 'KICT', 'KOHX', 'KSRX', 'KSJT']
#0C height + 1.5km
h_calstm = [5100, 5400, 4325, 4400, 4700, 5425]

In [3]:
storm_relative_dirstm = np.asarray([185])
# zdrlevs = np.asarray([3.25])
# kdplevs = np.asarray([1.5])
# REFlevs = np.asarray([45])
# REFlev1s = np.asarray([50])
# big_storms = np.asarray([300])
# zero_z_triggers = np.asarray([17])
# storm_to_tracks = np.asarray([0])
yearstm = np.asarray([2020])
monthstm = np.asarray([3])
daystm = np.asarray([3])
hourstm = np.asarray([6])
start_minstm = np.asarray([30])
durationstm = np.asarray([2.0])
#calibrations = np.asarray([-0.71])
h_calstm = np.asarray([2697])
calibrationstm = np.asarray([0.0])
stationstm = ['KOHX']

In [4]:
#Actual code for the ZDR calibration algorithm (more operational-ish version)
def multi_case_algorithm_ML1(year, month, day, hour, start_min, duration, station, h_cal):    
    #Settings
    #Here, set the initial time of the archived radar loop you want.
    #Changing this to pull in 15 minutes starting at the middle of the analysis period
    dt3 = datetime(year,month, day, hour, start_min) # Our specified time
    station = station
    dt = dt3 + timedelta(hours=duration/2.0)
    end_dt = dt + timedelta(hours=0.25)
    #Set up nexrad interface
    conn = nexradaws.NexradAwsInterface()
    scans = conn.get_avail_scans_in_range(dt,end_dt,station)
    #Change this to whatever folder you want the radar data to download into.
    results = conn.download(scans, 'ZDRCAL/'+str(year)+str(month)+str(day)+str(hour)+station)
    #Create times for possible file deletion later
    year1=year
    month1=month
    day1=day
    hour1=hour
    #query = rs.query()
    #Set the duration of the loop in hours
    #query.stations(station).time_range(dt, dt + timedelta(hours=duration))
    #cat = rs.get_catalog(query)
    #cat.datasets
    
    #Create an option for just reading all files in a folder
    #folder = 'Machine_Learning/LocalRadar/'+LocalFolder
    
    #Setting counters for figures and Pandas indices
    f = 27
    n = 1
    storm_index = 0
    scan_index = 0
    #Get calibration height
    h_oc = h_cal
    #Make lists for the calibrations
    zdr_cals = []
    zdr_raw = []
    cal_shapes = []
    #Create geod object for later distance and area calculations
    g = Geod(ellps='sphere')
    ####
    #Actual algorithm code starts here
    #for item in sorted(cat.datasets.items()):
    for i,scan in enumerate(results.iter_success(),start=1):
    #Local file option:
    #for radar_file in os.listdir(folder):
        #print(radar_file)
        #Loop over all files in the dataset and pull out each 0.5 degree tilt for analysi
        #ds = item[1]
        try:
            radar = scan.open_pyart()
        except:
            continue
        #Local file option
        #radar1 = pyart.io.nexrad_archive.read_nexrad_archive(folder+'/'+radar_file)
        #print('file read')
        time_start = netCDF4.num2date(radar.time['data'][0], radar.time['units'])
        object_number=0.0
        print(time_start)
        month = time_start.month
        if month < 10:
            month = '0'+str(month)
        hour = time_start.hour
        if hour < 10:
            hour = '0'+str(hour)
        minute = time_start.minute
        if minute < 10:
            minute = '0'+str(minute)
        day = time_start.day
        if day < 10:
            day = '0'+str(day)
        time_beg = time_start - timedelta(minutes=0.5)
        time_end = time_start + timedelta(minutes=0.5)
        sec_beg = time_beg.second
        sec_end = time_end.second
        min_beg = time_beg.minute
        min_end = time_end.minute
        h_beg = time_beg.hour
        h_end = time_end.hour
        d_beg = time_beg.day
        d_end = time_end.day
        if sec_beg < 10:
            sec_beg = '0'+str(sec_beg)
        if sec_end < 10:
            sec_end = '0'+str(sec_end)
        if min_beg < 10:
            min_beg = '0'+str(min_beg)
        if min_end < 10:
            min_end = '0'+str(min_end)
        if h_beg < 10:
            h_beg = '0'+str(h_beg)
        if h_end < 10:
            h_end = '0'+str(h_end)
        if d_beg < 10:
            d_beg = '0'+str(d_beg)
        if d_end < 10:
            d_end = '0'+str(d_end)
        print(datetime.utcnow())
        # mask out last 10 gates of each ray, this removes the "ring" around the radar.
        #Bring in all of the ungridded fields
        radar.fields['differential_reflectivity']['data'][:, -10:] = np.ma.masked
        cc_ungridded = radar.fields['cross_correlation_ratio']['data']
        ref_ungridded = radar.fields['reflectivity']['data']
        zdr_ungridded = radar.fields['differential_reflectivity']['data']
        print(zdr_ungridded.shape, 'ungridded zdr shape')
        #Get ungridded gate altitudes
        gate_altitude = radar.gate_altitude['data'][:]
        #Convert to height ARL
        h_agl = gate_altitude-radar.altitude['data'][0]
        #Convert to height relative to calibration level
        h_aoc = h_agl - h_oc

        #Now start masking out ZDR data
        zdr_c = zdr_ungridded
        zdr_c = ma.masked_where(ref_ungridded < 20, zdr_c)
        zdr_c = ma.masked_where(ref_ungridded > 35, zdr_c)
        zdr_c = ma.masked_where(cc_ungridded < 0.99, zdr_c)
        zdr_c = ma.masked_where(h_agl > (h_oc + 250), zdr_c)
        zdr_c = ma.masked_where(h_agl < (h_oc - 250), zdr_c)
        print(np.nanmax(zdr_ungridded), "ungridded zdr max")
        print(ma.MaskedArray.count(zdr_c))
        calshape = ma.MaskedArray.count(zdr_c)
        #Testing for elevation stuff
        alt_c = np.copy(h_agl)
        alt_c = ma.masked_where(ref_ungridded < 20, alt_c)
        alt_c = ma.masked_where(ref_ungridded > 35, alt_c)
        alt_c = ma.masked_where(cc_ungridded < 0.99, alt_c)
        alt_c = ma.masked_where(h_agl > (h_oc + 250), alt_c)
        alt_c = ma.masked_where(h_agl < (h_oc - 250), alt_c)

        #Get ungridded lats and lons
        ungrid_lons = radar.gate_longitude['data']
        ungrid_lats = radar.gate_latitude['data']
        cenlat = radar.latitude['data'][0]
        cenlon = radar.longitude['data'][0]
        
        #Get calibrations and raw means
        print(zdr_c.shape)
        print(zdr_c[~np.isnan(zdr_c)].shape)
        zdr_m = np.nanmean(zdr_c)
        zdr_ca = zdr_m - 0.15
        zdr_cals.append(zdr_ca)
        zdr_raw.append(zdr_m)
        cal_shapes.append(calshape)
        
            
    return zdr_cals, zdr_raw, cal_shapes

In [ ]:
#Actual code for the ZDR calibration algorithm (Local folder option)
def multi_case_algorithm_ML1_loc(year, month, day, hour, start_min, duration, station, h_cal, localfolder):    
    #Settings
    #Here, set the initial time of the archived radar loop you want.
    #Changing this to pull in 15 minutes starting at the middle of the analysis period
    dt3 = datetime(year,month, day, hour, start_min) # Our specified time
    station = station
    #dt = dt3 + timedelta(hours=duration/2.0)
    #end_dt = dt + timedelta(hours=0.25)
    #Set up nexrad interface
    #conn = nexradaws.NexradAwsInterface()
    #scans = conn.get_avail_scans_in_range(dt,end_dt,station)
    #Change this to whatever folder you want the radar data to download into.
    #results = conn.download(scans, 'ZDRCAL/'+str(year)+str(month)+str(day)+str(hour)+station)
    #Create times for possible file deletion later
    year1=year
    month1=month
    day1=day
    hour1=hour
    #query = rs.query()
    #Set the duration of the loop in hours
    #query.stations(station).time_range(dt, dt + timedelta(hours=duration))
    #cat = rs.get_catalog(query)
    #cat.datasets
    LocalFolder=localfolder
    #Create an option for just reading all files in a folder
    folder = 'Machine_Learning/LocalRadar/'+LocalFolder
    
    #Setting counters for figures and Pandas indices
    f = 27
    n = 1
    storm_index = 0
    scan_index = 0
    #Get calibration height
    h_oc = h_cal
    #Make lists for the calibrations
    zdr_cals = []
    zdr_raw = []
    cal_shapes = []
    #Create geod object for later distance and area calculations
    g = Geod(ellps='sphere')
    ####
    #Actual algorithm code starts here
    #for item in sorted(cat.datasets.items()):
    #for i,scan in enumerate(results.iter_success(),start=1):
    #Local file option:
    for radar_file in os.listdir(folder):
        #print(radar_file)
        #Loop over all files in the dataset and pull out each 0.5 degree tilt for analysi
        #ds = item[1]
        #radar = scan.open_pyart()
        #Local file option
        radar = pyart.io.nexrad_archive.read_nexrad_archive(folder+'/'+radar_file)
        #print('file read')
        time_start = netCDF4.num2date(radar.time['data'][0], radar.time['units'])
        object_number=0.0
        print(time_start)
        month = time_start.month
        if month < 10:
            month = '0'+str(month)
        hour = time_start.hour
        if hour < 10:
            hour = '0'+str(hour)
        minute = time_start.minute
        if minute < 10:
            minute = '0'+str(minute)
        day = time_start.day
        if day < 10:
            day = '0'+str(day)
        time_beg = time_start - timedelta(minutes=0.5)
        time_end = time_start + timedelta(minutes=0.5)
        sec_beg = time_beg.second
        sec_end = time_end.second
        min_beg = time_beg.minute
        min_end = time_end.minute
        h_beg = time_beg.hour
        h_end = time_end.hour
        d_beg = time_beg.day
        d_end = time_end.day
        if sec_beg < 10:
            sec_beg = '0'+str(sec_beg)
        if sec_end < 10:
            sec_end = '0'+str(sec_end)
        if min_beg < 10:
            min_beg = '0'+str(min_beg)
        if min_end < 10:
            min_end = '0'+str(min_end)
        if h_beg < 10:
            h_beg = '0'+str(h_beg)
        if h_end < 10:
            h_end = '0'+str(h_end)
        if d_beg < 10:
            d_beg = '0'+str(d_beg)
        if d_end < 10:
            d_end = '0'+str(d_end)
        print(datetime.utcnow())
        # mask out last 10 gates of each ray, this removes the "ring" around the radar.
        #Bring in all of the ungridded fields
        radar.fields['differential_reflectivity']['data'][:, -10:] = np.ma.masked
        cc_ungridded = radar.fields['cross_correlation_ratio']['data']
        ref_ungridded = radar.fields['reflectivity']['data']
        zdr_ungridded = radar.fields['differential_reflectivity']['data']
        print(zdr_ungridded.shape)
        #Get ungridded gate altitudes
        gate_altitude = radar.gate_altitude['data'][:]
        #Convert to height ARL
        h_agl = gate_altitude-radar.altitude['data'][0]
        #Convert to height relative to calibration level
        h_aoc = h_agl - h_oc

        #Now start masking out ZDR data
        zdr_c = zdr_ungridded
        zdr_c = ma.masked_where(ref_ungridded < 20, zdr_c)
        zdr_c = ma.masked_where(ref_ungridded > 35, zdr_c)
        zdr_c = ma.masked_where(cc_ungridded < 0.99, zdr_c)
        zdr_c = ma.masked_where(h_agl > (h_oc + 250), zdr_c)
        zdr_c = ma.masked_where(h_agl < (h_oc - 250), zdr_c)
        print(ma.MaskedArray.count(zdr_c))
        calshape = ma.MaskedArray.count(zdr_c)
        #Testing for elevation stuff
        alt_c = np.copy(h_agl)
        alt_c = ma.masked_where(ref_ungridded < 20, alt_c)
        alt_c = ma.masked_where(ref_ungridded > 35, alt_c)
        alt_c = ma.masked_where(cc_ungridded < 0.99, alt_c)
        alt_c = ma.masked_where(h_agl > (h_oc + 250), alt_c)
        alt_c = ma.masked_where(h_agl < (h_oc - 250), alt_c)

        #Get ungridded lats and lons
        ungrid_lons = radar.gate_longitude['data']
        ungrid_lats = radar.gate_latitude['data']
        cenlat = radar.latitude['data'][0]
        cenlon = radar.longitude['data'][0]
        
        #Get calibrations and raw means
        zdr_m = np.nanmean(zdr_c)
        zdr_ca = zdr_m - 0.15
        zdr_cals.append(zdr_ca)
        zdr_raw.append(zdr_m)
        cal_shapes.append(calshape)
            
    return zdr_cals, zdr_raw, cal_shapes

In [7]:
#Loop to run the actual algorithm
print(datetime.utcnow())
with open('Machine_Learning/ZDRCalsKOHX1.csv', mode='w') as cal_file:
    calWriter = csv.writer(cal_file, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
#You can either run all example cases (commented out line) or one at a time (runing line)
for i in range(len(durationstm)):
#for i in [0]:
    #zdr_cals, zdr_raw = multi_case_algorithm_ML1_loc(yearstm[i], monthstm[i], daystm[i], hourstm[i], start_minstm[i], durationstm[i], stationstm[i], h_calstm[i], localfolder[i])
    zdr_cals, zdr_raw, cal_s = multi_case_algorithm_ML1(yearstm[i], monthstm[i], daystm[i], hourstm[i], start_minstm[i], durationstm[i], stationstm[i], h_calstm[i])
    with open('Machine_Learning/ZDRCalsKOHX1.csv', mode='a') as cal_file:
                calWriter = csv.writer(cal_file, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
                calWriter.writerow([yearstm[i], monthstm[i], daystm[i], hourstm[i], start_minstm[i], np.nanmean(zdr_cals), np.nanmin(cal_s)])
print(datetime.utcnow())

2020-05-18 21:03:15.196408
Downloaded KOHX20200303_073349_V06
Downloaded KOHX20200303_074010_V06
2 out of 2 files downloaded...0 errors
2020-03-03 07:33:49.486021
2020-05-18 21:03:37.411329
(12600, 1832) ungridded zdr shape
7.9375 ungridded zdr max
23176
(12600, 1832)
(23083200,)
2020-03-03 07:40:10.282990
2020-05-18 21:03:50.456489
(12600, 1832) ungridded zdr shape
7.9375 ungridded zdr max
24861
(12600, 1832)
(23083200,)
2020-05-18 21:03:58.247874


In [6]:
print(np.mean(zdr_cals))

-0.11823112312704326
